In [ ]:
!pip install faiss-cpu --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.9 MB/s eta 0:00:00


In [ ]:
import json
import time
import os
import numpy as np
import pickle
import faiss
import requests
from tqdm import tqdm
from google.colab import userdata

In [ ]:
# Configurações
MISTRAL_API_KEY = userdata.get("MISTRAL_KEY")  # Substitua pela sua chave
INPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/projeto II V3/scrapyFiles/biblioteca_dados_combinado.json"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/projeto II V3/scrapyFiles/faiss_index"
EMBEDDING_DIM = 1024  # Dimensão dos embeddings do modelo Mistral
BASE_WAIT_TIME = 1.1  # Tempo base de espera entre requisições
MAX_RETRIES = 5      # Número máximo de tentativas para cada livro

# Criar diretório de saída, se não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Função para gerar embeddings usando a API da Mistral com retry e backoff
def get_embedding_with_retry(text, api_key=MISTRAL_API_KEY, max_retries=MAX_RETRIES):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    url = "https://api.mistral.ai/v1/embeddings"
    payload = {
        "model": "mistral-embed",
        "input": text
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(url, json=payload, headers=headers)

            if response.status_code == 200:
                return response.json()["data"][0]["embedding"]
            elif response.status_code == 429:
                wait_time = BASE_WAIT_TIME * (2 ** attempt)
                print(f"Rate limit excedido. Tentativa {attempt+1}/{max_retries}. Aguardando {wait_time:.2f} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Erro na API: {response.status_code}")
                print(response.text)
                time.sleep(BASE_WAIT_TIME)
        except Exception as e:
            print(f"Exceção ocorreu: {str(e)}")
            time.sleep(BASE_WAIT_TIME)

    print(f"Falha após {max_retries} tentativas. Pulando este livro.")
    return None


In [ ]:
# Função para formatar o texto de cada livro
def format_book_text(book):
    if not isinstance(book, dict):
        print(f"Erro: Livro inválido: {book}")
        return ""
    titulo = book.get("titulo", "")
    autor = book.get("autor", "")
    coautor = book.get("co-autor", "")
    idioma = book.get("idioma", "")
    pais = book.get("pais", "")
    itype = book.get("itype", "")
    nome_comum = ", ".join(book.get("nome_comum", [])) if isinstance(book.get("nome_comum"), list) else ""
    shelvingloc = book.get("shelvingloc", "")
    call_no = book.get("call_no", "")

    text = f"""
Título: {titulo}
Autor: {autor}
Co-autor: {coautor}
Idioma: {idioma}
País: {pais}
Tipo: {itype}
Assuntos: {nome_comum}
Localização: {shelvingloc}
Cota: {call_no}
"""
    return text.strip()


In [ ]:
# Carregar dados do JSON
print("Carregando dados do arquivo JSON...")
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    books = json.load(f)

print(f"Total de livros carregados: {len(books)}")

# Carregar progresso parcial, se existir
embeddings = []
metadata = []
try:
    embeddings_array = np.load(os.path.join(OUTPUT_DIR, "book_embeddings_temp.npy"))
    with open(os.path.join(OUTPUT_DIR, "book_metadata_temp.pkl"), "rb") as f:
        metadata = pickle.load(f)
    embeddings = embeddings_array.tolist()
    processed_ids = {m["id"] for m in metadata}
    books = [b for b in books if b.get("id") not in processed_ids]
    print(f"Carregado progresso parcial: {len(embeddings)} livros já processados.")
except FileNotFoundError:
    print("Nenhum progresso parcial encontrado. Iniciando do zero.")

# Gerar embeddings para cada livro
print("Gerando embeddings para cada livro com backoff...")
try:
    for book in tqdm(books, total=len(books)):
        book_text = format_book_text(book)
        embedding = get_embedding_with_retry(book_text)

        if embedding:
            embeddings.append(embedding)
            book_metadata = {
                "id": book.get("id", ""),
                "titulo": book.get("titulo", ""),
                "autor": book.get("autor", ""),
                "idioma": book.get("idioma", ""),
                "nome_comum": book.get("nome_comum", []),
                "shelvingloc": book.get("shelvingloc", ""),
                "call_no": book.get("call_no", "")
            }
            metadata.append(book_metadata)

            # Salvamento incremental a cada 100 livros
            if len(embeddings) % 100 == 0 and len(embeddings) > 0:
                np.save(os.path.join(OUTPUT_DIR, "book_embeddings_temp.npy"), np.array(embeddings).astype('float32'))
                with open(os.path.join(OUTPUT_DIR, "book_metadata_temp.pkl"), "wb") as f:
                    pickle.dump(metadata, f)
                print(f"Checkpoint salvo: {len(embeddings)} livros processados.")

        time.sleep(BASE_WAIT_TIME)
except KeyboardInterrupt:
    print("\nProcesso interrompido pelo usuário.")
    if len(embeddings) > 0:
        print("Salvando progresso parcial...")
        np.save(os.path.join(OUTPUT_DIR, "book_embeddings_temp.npy"), np.array(embeddings).astype('float32'))
        with open(os.path.join(OUTPUT_DIR, "book_metadata_temp.pkl"), "wb") as f:
            pickle.dump(metadata, f)
        print(f"Progresso parcial salvo: {len(embeddings)} livros processados.")
    raise

# Converter lista de embeddings para numpy array e salvar
if len(embeddings) > 0:
    print(f"Embeddings gerados: {len(embeddings)}")
    embeddings_array = np.array(embeddings).astype('float32')
    print(f"Dimensão de cada embedding: {embeddings_array.shape[1]}")

    # Criar índice FAISS
    print("Criando índice FAISS...")
    dimension = embeddings_array.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_array)

    # Salvar os arquivos finais
    print("Salvando embeddings, metadados e índice FAISS...")
    np.save(os.path.join(OUTPUT_DIR, "book_embeddings.npy"), embeddings_array)
    with open(os.path.join(OUTPUT_DIR, "book_metadata.pkl"), "wb") as f:
        pickle.dump(metadata, f)
    faiss.write_index(index, os.path.join(OUTPUT_DIR, "book_index.faiss"))

    print(f"Processo concluído! Arquivos salvos em: {OUTPUT_DIR}")
    print("Arquivos gerados:")
    print("- book_embeddings.npy: Array NumPy contendo todos os embeddings")
    print("- book_metadata.pkl: Arquivo pickle com metadados correspondentes")
    print("- book_index.faiss: Índice FAISS para busca de similaridade")
else:
    print("Nenhum embedding foi gerado com sucesso. Nenhum arquivo foi salvo.")

Carregando dados do arquivo JSON...
Total de livros carregados: 18266
Carregado progresso parcial: 18200 livros já processados.
Gerando embeddings para cada livro com backoff...


  2%|▏         | 1/66 [00:01<01:49,  1.68s/it]

Rate limit excedido. Tentativa 1/5. Aguardando 1.10 segundos...


 27%|██▋       | 18/66 [00:26<01:09,  1.44s/it]

Rate limit excedido. Tentativa 1/5. Aguardando 1.10 segundos...


...

Rate limit excedido. Tentativa 1/5. Aguardando 1.10 segundos...


 98%|█████████▊| 65/66 [01:46<00:01,  1.64s/it]

Rate limit excedido. Tentativa 1/5. Aguardando 1.10 segundos...


100%|██████████| 66/66 [01:49<00:00,  1.65s/it]


Embeddings gerados: 18266
Dimensão de cada embedding: 1024
Criando índice FAISS...
Salvando embeddings, metadados e índice FAISS...
Processo concluído! Arquivos salvos em: /content/drive/MyDrive/Colab Notebooks/projeto II V3/scrapyFiles/faiss_index
Arquivos gerados:
- book_embeddings.npy: Array NumPy contendo todos os embeddings
- book_metadata.pkl: Arquivo pickle com metadados correspondentes
- book_index.faiss: Índice FAISS para busca de similaridade
